# Jupyter Notebooks - part 1
* ### Cells and magics 

In this lesson you will learn 
- how *markdown* and *code* cells work
- how to use keyboard shortcuts to speed up your work
- how to take advantage of notebook *magics* 
- how to create new custom magic commands
- how to plot interactively
- how to mix in different markup and programming languages (html, LaTeX, bash, ruby, perl, R, octave)

## Cells

- *code cells* contain code to be interpreted by the *kernel* (Python, R, Julia, Octave/Matlab...)
- *markdown cells* contain formatted text written in Markdown 

Let's give it a try

### <font color="red"> *Exercise 1.1* </font>

* Spend 5-10 minutes playing around and familiarizing yourself with the Jupyter Notebook
* If you prefer R, Matlab/Octave, Julia, or any other language, and you have installed the corresponding kernel, you can switch kernels or create a new notebook 

### Keyboard shortcuts 
* `Enter` key to enter Edit mode (`Escape` to enter Command mode)
* `Ctrl`-`Enter`: run the cell
* `Shift`-`Enter`: run the cell and select the cell below
* `Alt`-`Enter`: run the cell and insert a new cell below
* `Ctrl`-`s`: save the notebook
* `Tab` key for code completion or indentation
* `m` and `y` to toggle between Markdown and Code cells
* `d-d` to delete a cell
* `z` to undo deleting
* `a/b` to insert cells above/below current cell
* `x/c/v` to cut/copy/paste cells
* `Up/Down` or `k/j` to select previous/next cells
* `h` for help menu for keyboard shortcuts
* Append `?` for help on commands/methods, `??` to show source 

### Shell commands
  - Many common linux shell commands are available as magics: %ls, %pwd, %mkdir, %cp, %mv, %cd, *etc.*
  - Other shell commands are available by prepending with !
  - Useful, e.g., for managing the python environment


In [ ]:
!echo "hello"

In [ ]:
!pip list

## Jupyter magics

Magics are a simple command language which significantly extend the power of Jupyter 

Two kinds of magics:

  - **Line magics**: commands prepended by one % character and whose arguments only extend to the end of the current line.
  - **Cell magics**: use two percent characters as a marker (%%), receive as argument the whole cell (must be used as the first line in a cell)

Other features:
  - Use %lsmagic magic to list all available line and cell magics
  - Question mark shows help: `%lsmagic?`
  - Default behavior: `Automagic is ON, % prefix IS NOT needed for line magics`
  - Additional magics can be created

In [ ]:
%lsmagic

In [ ]:
%magic

You can capture the output of magic (and shell) commands

In [ ]:
ls_out = %ls
ls_out

In [ ]:
%sx?

In [ ]:
ls_out = %sx ls
ls_out

In [ ]:
ls_out = !ls
ls_out

### %timeit
- Timing execution

In [ ]:
%timeit import time ; time.sleep(1)

In [ ]:
import numpy as np

In [ ]:
%%timeit 
a = np.random.rand(100, 100)
np.linalg.eigvals(a)

### %capture
- this magic captures the stdout/stderr of any code cell, either to discard it or to store it in a variable for later use

In [ ]:
%%capture output
%ls

In [ ]:
output.stdout, output.stderr    

### %%writefile
- writes the cell contents as a named file

In [ ]:
%%writefile foo.py
print('Hello world')

### %run 
 - executes python code from .py files 
 - can also execute other jupyter notebooks

In [ ]:
%run foo

### %load
 - loads code directly into cell. File either from local disk or from the internet
 - After uncommenting the code below and executing, it will replace the content of cell with contents of file.

In [ ]:
# %load http://matplotlib.org/mpl_examples/pylab_examples/contour_demo.py

### %pastebin
- upload code to Github's Gist paste bin, returning the URL

In [ ]:
%pastebin foo.py

### %bookmark 

- Manage IPython's bookmark system.
- Examples:
    ```
    %bookmark <name>       - set bookmark to current dir
    %bookmark <name> <dir> - set bookmark to <dir>
    %bookmark -l           - list all bookmarks
    %bookmark -d <name>    - remove bookmark
    %bookmark -r           - remove all bookmarks```
- You can later on access a bookmarked folder with

    ```
    %cd -b <name>
    ```

### %env
Manage environment variables of your notebook without restarting the jupyter server process. Some libraries use environment variables to control behavior, and %env is the most convenient way.

In [ ]:
%env OMP_NUM_THREADS=4

### %store

Pass variables between notebooks

To access stored variable in new notebook:

```
%store -r data
print data
```

In [ ]:
data = 'string to pass to a different notebook'
%store data
del data # This has deleted the variable

### %debug
* activate interactive debugger

In [ ]:
%debug?

In [ ]:
# %load debug_example.py

Don't forget to exit the debugger!

### %prun
 - Python code profiler

### <font color="red"> *Exercise 1.2* </font>

* load the file random_walk.py into a cell below
* add the prun *cell* magic
* run the cell and view the output
* can you make a simple adjustment to speed up the code?

### <font color="green"> *Solution* </font>

> The solution can be found in the solutions.ipynb notebook

### Installing a new magic command

We will now install a line-profiler to get more detailed profile, and hopefully find ways to speed up the code

In [ ]:
# %load random_walk.py

In [ ]:
# !pip install line_profiler

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f main main()

### Creating your own custom magic

Using the `@register_cell_magic` decorator, we will create a cell magic command that compiles C++ code and executes it


> This example has been borrowed from the [IPython Minibook](http://ipython-books.github.io/), by Cyrille Rossant, Packt Publishing, 2015.


In [ ]:
from IPython.core.magic import register_cell_magic

In [ ]:
@register_cell_magic
def cpp(line, cell):
    """Compile, execute C++ code, and return the standard output."""

    # We first retrieve the current IPython interpreter instance.
    ip = get_ipython()

    # We define the source and executable filenames.
    source_filename = '_temp.cpp'
    program_filename = '_temp'

    # We write the code to the C++ file.
    with open(source_filename, 'w') as f:
        f.write(cell)

    # We compile the C++ code into an executable.
    compile = ip.getoutput("g++ {0:s} -o {1:s}".format(
        source_filename, program_filename))

    # We execute the executable and return the output.
    output = ip.getoutput('./{0:s}'.format(program_filename))

    print('\n'.join(output))

In [ ]:
%%cpp 
#include<iostream>
int main(){
    std::cout << "Hello World";
}

This cell magic is now only available in the current notebook. To make it permanent we need to make an IPython extension. This we do by writing the definition of the function `cpp()` into a file on `PYTHONPATH` (for example current directory), and add a small function at the end

In [ ]:
%%writefile cpp_ext.py
def cpp(line, cell):
    """Compile, execute C++ code, and return the standard output."""

    # We first retrieve the current IPython interpreter instance.
    ip = get_ipython()

    # We define the source and executable filenames.
    source_filename = '_temp.cpp'
    program_filename = '_temp'

    # We write the code to the C++ file.
    with open(source_filename, 'w') as f:
        f.write(cell)

    # We compile the C++ code into an executable.
    compile = ip.getoutput("g++ {0:s} -o {1:s}".format(
        source_filename, program_filename))

    # We execute the executable and return the output.
    output = ip.getoutput('./{0:s}'.format(program_filename))

    print('\n'.join(output))

def load_ipython_extension(ipython):
    ipython.register_magic_function(cpp,'cell')

In [ ]:
%load_ext cpp_ext

In [ ]:
%%cpp?

### Interactive plotting

In [ ]:
%matplotlib?

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
x = np.linspace(0,2*np.pi,100)
y = np.sin(x)
plt.plot(x,y, 'r-')
plt.show()

### More interactivity using widgets

In [ ]:
from ipywidgets import interact # IPython.html.widgets before IPython 4.0

@interact
def plot(n=(1,6)):
    x = np.linspace(0,2*np.pi,100)
    y = np.sin(n*x)
    plt.plot(x,y, 'r-')
    plt.show()

### <font color="red"> *Exercise 1.3* </font>

- Create a widget that computes the square of integers between 0 and 10!

### <font color="green"> *Solution* </font>

> The solution can be found in the solutions.ipynb notebook

### Further shell access with %%bash magic
 - Run cells with bash in a subprocess.

In [ ]:
%%bash
mkdir tmpdir
cd tmpdir
pwd
echo "foo" > test.file
ls
cat test.file
cd ..
rm -r tmpdir

In [ ]:
%%bash
echo "hi, stdout"
echo "hello, stderr" >&2

In [ ]:
%%bash --out output --err error
echo "hi, stdout"
echo "hello, stderr" >&2

In [ ]:
print(error)
print(output)

### Mixing in other languages (given that they're installed)

The `%%script` magic is like the #! (shebang) line of script,
specifying a program (bash, perl, ruby, etc.) with which to run.  
But one can also directly use these:
- %%ruby
- %%perl
- %%bash
- %%html
- %%latex
- %%R

In [ ]:
%%ruby
puts 'Hi, this is ruby.'

In [ ]:
%%script ruby
puts 'Hi, this is also ruby.'

In [ ]:
%%perl
print "Hello, this is perl\n";

In [ ]:
%%bash
echo "Hullo, I'm bash"

In [ ]:
%%html
<table>
<tr>
<th>Header 1</th>
<th>Header 2</th>
</tr>
<tr>
<td>row 1, cell 1</td>
<td>row 1, cell 2</td>
</tr>
<tr>
<td>row 2, cell 1</td>
<td>row 2, cell 2</td>
</tr>
</table>

In [ ]:
%%latex
\begin{align}
\nabla \times \vec{\mathbf{B}} -\, \frac1c\, \frac{\partial\vec{\mathbf{E}}}{\partial t} & = \frac{4\pi}{c}\vec{\mathbf{j}} \\
\nabla \cdot \vec{\mathbf{E}} & = 4 \pi \rho \\
\nabla \times \vec{\mathbf{E}}\, +\, \frac1c\, \frac{\partial\vec{\mathbf{B}}}{\partial t} & = \vec{\mathbf{0}} \\
\nabla \cdot \vec{\mathbf{B}} & = 0
\end{align}

### R

The R world already has a powerful IDE, RStudio, where one can annotate code using Markdown and export to HTML.  
A key difference between RStudio and Jupyter is that in Jupyter one can modify and rerun individual cells, without having to rerun everything.

For R, need to load the rpy2 IPython extension

In [ ]:
# first we need to install the necessary packages
# !conda install -c r r-essentials

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
myString <- "Hello, this is R"
print ( myString)

Inline plotting in R is straightforward 

In [ ]:
%%R 
# Define the cars vector with 5 values
cars <- c(1, 3, 6, 4, 9)

# Graph cars using blue points overlayed by a line 
plot(cars, type="o", col="blue")

# Create a title with a red, bold/italic font
title(main="Autos", col.main="red", font.main=4)

Data in R cells is of course persistent

In [ ]:
%%R 
barplot(cars)

We can plot a Python pandas dataframe with R code

In [ ]:
import pandas as pd
df = pd.DataFrame({
    'cups_of_coffee': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    'productivity': [2, 5, 6, 8, 9, 8, 0, 1, 0, -1]
})

In [ ]:
%%R -i df -w 6 -h 4 --units cm -r 200
# the first line says 'import df and make default figure size 5 by 5 inches 
# with resolution 200. You can change the units to px, cm, etc. as you wish.
library(ggplot2)
ggplot(df, aes(x=cups_of_coffee, y=productivity)) + geom_line()

### Octave/Matlab

We switch to an Octave kernel. This comes with a new set of magics

In [ ]:
%lsmagic

In [ ]:
t = linspace(0,6*pi,100);
plot(t,sin(t))
grid on
hold on
plot(t,cos(t), 'r')

In [ ]:
tx = ty = linspace (-8, 8, 41)';
[xx, yy] = meshgrid (tx, ty);
r = sqrt (xx .^ 2 + yy .^ 2) + eps;
tz = sin (r) ./ r;
mesh (tx, ty, tz);

### Julia

> This example has been adapted from the [IPython Cookbook](http://ipython-books.github.io/), by Cyrille Rossant, Packt Publishing, 2014.


[Julia](https://en.wikipedia.org/wiki/Julia_(programming_language) is a high-level dynamic language designed for high-performance numerical analysis and computional science.

We switch to the Julia kernel.

In [ ]:
println("Hello world!")

We create a polymorphic function `f`. It will be evaluated on arrays, so we use elementwise operators with a dot (.) prefix

In [ ]:
f(z, c) = z.*z .+ c

Create a function julia that computes a Julia set. Optional named arguments are separated from positional arguments by a semicolon (;). Julia's syntax for flow control is close to Python's, except that colons are dropped, indentation doesn't count, and block end keywords are mandatory

In [ ]:
function julia(z, c; maxiter=200)
    for n = 1:maxiter
        if abs2(z) > 4.0
            return n-1
        end
        z = f(z, c)
    end
    return maxiter
end

We can use Python packages from Julia. PyCall is first installed by Julia's built-in package manager (Pkg)

In [ ]:
#Pkg.add("PyCall")
using PyCall

Python packages can be imported with the `@pyimport` **macro** (a metaprogramming feature in Julia). This macro is the equivalent of Python's `import` command.

In [ ]:
@pyimport numpy as np

The Gadfly plotting library offers a high-level plotting interface

In [ ]:
#Pkg.add("Gadfly")
using Gadfly

We compute a Julia set by using two nested loops. Unlike Python, there is no significant performance penalty using for loops instead of vectorized operations in Julia. High-performance code can be written either with vectorized operations or for loops.

In [ ]:
@time m = [julia(complex(r, i), complex(-0.06, 0.67)) 
#@time m = [julia(complex(r, i), complex(-0.8, 0.156)) 
           for i = 1:-.001:-1,
               r = -1.5:.001:1.5];

Use the `PyPlot` package to draw matplotlib figures in Julia.

In [ ]:
#Pkg.add("PyPlot")
using PyPlot

In [ ]:
imshow(m, cmap="RdGy", 
       extent=[-1.5, 1.5, -1, 1]);